In [1]:
import pandas as pd
import numpy as np
import os
import sys

In [4]:
# Combining reviews

with open('../googleplay-apps.txt') as appfile:
    line = appfile.readline()

    while(line):
        app = line.strip()

        applestoredf = pd.read_csv('./review-files-python/applestore-review-' + app + '_sentiment_python.csv')
        googleplaydf = pd.read_csv('./review-files-python/googleplay-review-' + app + '_sentiment_python.csv')
        googleplaydf = googleplaydf.rename(columns={"content": "review", "score": "rating", "at": "date"})
        cdf = pd.concat([applestoredf, googleplaydf], ignore_index=True)

        cdf.to_csv('review-files-combined-python/combined-review-' + app + '_sentiment.csv', index=False)
        line = appfile.readline()

In [5]:
# Standardize reviews

import string
import re
from unidecode import unidecode
import nltk
from nltk.corpus import stopwords

def standardize(text):
    # Convert to lowercase
    text = text.lower()
    
    # Remove leading/trailing whitespace
    text = text.strip()
    
    # Remove punctuation
    text = re.sub(r'[^\w\s]', '', text)
    
    # Remove diacritics and accents
    text = unidecode(text)

    # Tokenization
    text = text.split()

    # Remove stopwords
    stop_words = set(stopwords.words("english"))
    text = [word for word in text if word not in stop_words]

    # Return the standardized string
    return ' '.join(text)


filelist = [file for file in os.listdir('review-files-combined-python/')]


for file in filelist:
    print(file)

    df = pd.read_csv('review-files-combined-python/' + file)
    df['review'] = df['review'].astype('str')

    df['standardizedReview'] = df['review'].apply(standardize)

    output_file = os.path.splitext(file)[0] + '_standardized.csv'

    df.to_csv('review-files-combined-standardized-python/' + output_file, index=False)
    

combined-review-datacamp_sentiment.csv
combined-review-learn-python-programiz_sentiment.csv
combined-review-programming-hub_sentiment.csv
combined-review-codeacademy_sentiment.csv
combined-review-sololearn_sentiment.csv
combined-review-programming-hero_sentiment.csv
combined-review-encode_sentiment.csv
combined-review-mimo_sentiment.csv


In [6]:
# Getting stems

from stemming.porter2 import stem
from collections import defaultdict

with open('features.txt') as file:
    str = file.read()

features = [feature.strip() for feature in str.split(',')]

df = pd.read_csv('features-all.csv')
print(df)

def stem_phrase(phrase):
    words = phrase.split()
    stemmed_words = [stem(word) for word in words]
    return ' '.join(stemmed_words)

df['Stem'] = df['Feature'].apply(stem)

df['SynonymStem'] = df['Synonym'].apply(stem)
df['SynonymStem'] = df['SynonymStem'].apply(standardize)

print(df)

df.to_csv('feature-list.csv', index=False)

my_dict = defaultdict(set)
for feature, synonym in zip(df['Feature'], df['SynonymStem']):
    my_dict[feature].add(synonym)

# Print the resulting dictionary
print(my_dict)

              Feature                 Synonym              Stem   
0       daily streaks           daily streaks      daily streak  \
1       daily streaks  consecutive engagement      daily streak   
2       daily streaks           habit-forming      daily streak   
3       daily streaks           daily rewards      daily streak   
4       daily streaks           daily bonuses      daily streak   
..                ...                     ...               ...   
375  status quo sloth          accountability  status quo sloth   
376  status quo sloth            leaderboards  status quo sloth   
377  status quo sloth       progress tracking  status quo sloth   
378  status quo sloth    long-term engagement  status quo sloth   
379  status quo sloth     sustained retention  status quo sloth   

           SynonymStem  
0         daily streak  
1    consecutive engag  
2            habitform  
3         daily reward  
4          daily bonus  
..                 ...  
375            accou

In [7]:
# Filtering reviews

filelist = [file for file in os.listdir('review-files-combined-standardized-python/')]


for file in filelist:
    print(file)

    df = pd.read_csv('review-files-combined-standardized-python/' + file)
    df['standardizedReview'] = df['standardizedReview'].astype('str')
    

    df['mentionedFeature'] = df['standardizedReview'].apply(lambda x: ', '.join([feature for feature, synonyms in my_dict.items() if any(syn in x for syn in synonyms)]))
    df['mentionedSynonyms'] = df['standardizedReview'].apply(lambda x: ', '.join([syn for feature, synonyms in my_dict.items() for syn in synonyms if syn in x]))

    df.replace('', np.nan, inplace=True)
    df.dropna(subset=['mentionedFeature'], inplace=True)
    #df['standardizedReview'] = df['review'].apply(standardize)

    output_file = os.path.splitext(file)[0] + '_filtered.csv'

    df.to_csv('review-files-combined-filtered-python/' + output_file, index=False)

combined-review-mimo_sentiment_standardized.csv
combined-review-datacamp_sentiment_standardized.csv
combined-review-programming-hero_sentiment_standardized.csv
combined-review-learn-python-programiz_sentiment_standardized.csv
combined-review-programming-hub_sentiment_standardized.csv
combined-review-encode_sentiment_standardized.csv
combined-review-sololearn_sentiment_standardized.csv
combined-review-codeacademy_sentiment_standardized.csv


In [9]:
# Aggregating feature sentiments

scores = defaultdict(list)
for feature in my_dict.keys():
    feature = feature.lower()
    scores[feature] = [0, 0]

print(scores)
print(scores['daily streaks'][0])
print(scores['daily streaks'][1])

filelist = [file for file in os.listdir('review-files-combined-filtered-python/')]

for file in filelist:
    print(file)

    df = pd.read_csv('review-files-combined-filtered-python/' + file)

    for index, row in df.iterrows():
        mentioned_features = [feature.strip() for feature in row['mentionedFeature'].split(',')]
        #print(mentioned_features)
        for feature in mentioned_features:
            #print(feature)
            if row['sentiment'] == 'POSITIVE':
                scores[feature][0] += 1
            elif row['sentiment'] == 'NEGATIVE':
                scores[feature][1] -= 1

for feature in scores.keys():
    if scores[feature][0] - scores[feature][1] == 0:
        scores[feature].append(0)
        continue
    scores[feature].append((scores[feature][0] + scores[feature][1]) / (scores[feature][0] - scores[feature][1]))

print(scores)

overallscoredf = pd.DataFrame(columns = ['feature', 'positiveCount', 'negativeCount', 'score'])

for feature in scores.keys():
    new_row = pd.Series({'feature': feature, 'positiveCount': scores[feature][0], 'negativeCount': scores[feature][1], 'score': scores[feature][2]})
    overallscoredf = pd.concat([overallscoredf, new_row.to_frame().T], ignore_index=False)

overallscoredf.to_csv('feature-scores_python.csv')

defaultdict(<class 'list'>, {'daily streaks': [0, 0], 'points system': [0, 0], 'progress bar': [0, 0], 'strike habit/competition': [0, 0], 'social discovery': [0, 0], 'challenges': [0, 0], 'leaderboard/competition': [0, 0], 'boosters': [0, 0], 'virtual economy': [0, 0], 'torture breaks': [0, 0], 'visual grave': [0, 0], 'high five': [0, 0], 'last mile drive': [0, 0], 'anticipation parade': [0, 0], 'avatar': [0, 0], 'win states': [0, 0], 'voting': [0, 0], 'crowning': [0, 0], 'status quo sloth': [0, 0]})
0
0
combined-review-mimo_sentiment_standardized_filtered.csv
combined-review-sololearn_sentiment_standardized_filtered.csv
combined-review-datacamp_sentiment_standardized_filtered.csv
combined-review-codeacademy_sentiment_standardized_filtered.csv
combined-review-programming-hero_sentiment_standardized_filtered.csv
combined-review-learn-python-programiz_sentiment_standardized_filtered.csv
combined-review-encode_sentiment_standardized_filtered.csv
combined-review-programming-hub_sentiment_

In [10]:
# Feature sentiment per app

scoredf = pd.DataFrame(columns = ['app', 'feature', 'positiveCount', 'negativeCount', 'score'])

filelist = [file for file in os.listdir('review-files-combined-filtered-python/')]

with open('../googleplay-apps.txt') as appfile:
    line = appfile.readline()

    while(line):
        app = line.strip()
        
        df = pd.read_csv('review-files-combined-filtered-python/combined-review-' + app + '_sentiment_standardized_filtered.csv')

        appscores = defaultdict(list)
        for feature in my_dict.keys():
            feature = feature.lower()
            appscores[feature] = [0, 0]

        for index, row in df.iterrows():
            mentioned_features = [feature.strip() for feature in row['mentionedFeature'].split(',')]
            #print(mentioned_features)
            for feature in mentioned_features:
                #print(feature)
                if row['sentiment'] == 'POSITIVE':
                    appscores[feature][0] += 1
                elif row['sentiment'] == 'NEGATIVE':
                    appscores[feature][1] -= 1

        for feature in appscores.keys():
            if appscores[feature][0] - appscores[feature][1] == 0:
                appscores[feature].append(0)
                continue
            appscores[feature].append((appscores[feature][0] + appscores[feature][1]) / (appscores[feature][0] - appscores[feature][1]))

        for feature in appscores.keys():
            new_row = pd.Series({'app': app, 'feature': feature, 'positiveCount': appscores[feature][0], 'negativeCount': appscores[feature][1], 'score': appscores[feature][2]})
            
            scoredf = pd.concat([scoredf, new_row.to_frame().T], ignore_index=False)

        line = appfile.readline()

scoredf.to_csv('feature-scores-by-app_python.csv')
